In [397]:
#importin the libraries

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import Window

In [398]:
#create spark session
spark = SparkSession \
        .builder \
        .appName("LOL Mini Project") \
        .getOrCreate()

In [399]:
#Imprtint the data
raw_df = spark.read.json('matches.json')

In [400]:
raw_df.show(1)

+--------------------+--------------------+
|                info|            metadata|
+--------------------+--------------------+
|{1670799555338, 9...|{2, BR1_264293105...|
+--------------------+--------------------+
only showing top 1 row



Champion win Rate

In [401]:
d = raw_df.selectExpr('inline(arrays_zip(info.participants.championName,info.participants.win))')


win_df= d.groupBy('championName','win').count().filter( d.win == 'true')

all_df= d.groupBy('championName').count().alias('all').selectExpr('championName', 'count as match_count')

rate_df = win_df.join(all_df, win_df.championName==all_df.championName, 'outer').fillna(0)

win_rate_df = rate_df.withColumn('winrate', (col('count') / col('match_count')) * 100 ).selectExpr('all.championName as championName','winrate')

win_rate_df=win_rate_df.orderBy(win_rate_df['winrate'].desc())


Champion Ban Rate

In [402]:
ban_df= raw_df.selectExpr('explode(info.teams) as m' ,'explode(m.bans) as s' , 's.championId as id')\
    .groupBy('id').count().sort('count')

match_count = raw_df.select('metadata').count()

ban_df = ban_df.withColumn('Ban Rate', (col('count') / match_count ) * 100)

ban_df=ban_df.orderBy(ban_df['Ban Rate'].desc())


Champion Pick Rate

In [403]:
pick_df= raw_df.selectExpr('explode(info.participants.championName) as championName').groupBy('championName').count()
pick_rate_df= pick_df.withColumn('pick Rate', (col('count') / match_count ) * 100)

pick_rate_df=pick_rate_df.orderBy(pick_rate_df['pick Rate'].desc())


Champion Synergies or duos

In [404]:
def get_pairs(x):
    return [(x[1][p1], x[1][p2]) for p1 in range(len(x[1])) for p2 in range(p1+1,len(x[1]))]

In [413]:
s= raw_df.selectExpr('inline(arrays_zip(info.participants.championName,info.participants.win))', 'metadata.matchId')

Champion_Synergies_win_RDD= s.filter(s.win=='true')\
    .rdd.map(lambda x: (x[2], x[0]))\
    .reduceByKey(lambda x,y: x + ' ' + y)\
    .map(lambda x: (x[0],x[1].split()))\
    .flatMap(lambda x: get_pairs(x))\
    .map(lambda x: (x , 1))\
    .reduceByKey(lambda x,y : x + y)

Champion_Synergies_RDD= s.rdd.map(lambda x: (x[2], x[0]))\
    .reduceByKey(lambda x,y: x + ' ' + y)\
    .map(lambda x: (x[0],x[1].split()))\
    .flatMap(lambda x: get_pairs(x))\
    .map(lambda x: (x , 1))\
    .reduceByKey(lambda x,y : x + y)

Champion_Synergies_win_df = Champion_Synergies_win_RDD.toDF()
Champion_Synergies_df= Champion_Synergies_RDD.toDF()
Champion_Synergies_df= Champion_Synergies_df.withColumnRenamed(Champion_Synergies_df.columns[0], 'duo' )
Champion_Synergies_df= Champion_Synergies_df.withColumnRenamed(Champion_Synergies_df.columns[1],'all_matches')
Champion_Synergies_win_df= Champion_Synergies_win_df.withColumnRenamed(Champion_Synergies_win_df.columns[0], 'duo2' )
Champion_Synergies_win_df= Champion_Synergies_win_df.withColumnRenamed(Champion_Synergies_win_df.columns[1],'win')

best_Champion_Synergies_df = Champion_Synergies_df.join(Champion_Synergies_win_df\
     , Champion_Synergies_df.duo == Champion_Synergies_win_df.duo2, 'outer' ).fillna(0)

best_Champion_Synergies_df = best_Champion_Synergies_df.withColumn('win_rate', (col('win') / col('all_matches') * 100 ))\
    .select('duo','win_rate')
best_Champion_Synergies_df=best_Champion_Synergies_df.orderBy(best_Champion_Synergies_df['win_rate'].desc())

best_Champion_Synergies_df= best_Champion_Synergies_df.withColumn('dues', best_Champion_Synergies_df.duo.cast(StringType()))\
    .selectExpr('dues','win_rate')

Item win Rate

In [406]:

items_df = raw_df.selectExpr('''inline(arrays_zip(info.participants.item0,
                        info.participants.item1,info.participants.item2, info.participants.item3,
                        info.participants.item4, info.participants.item5,
                        info.participants.item6,info.participants.win))''')


items_df = items_df.selectExpr('''stack(7,boolean(win),item0,boolean(win),item1,
                  boolean(win),item2,boolean(win),item3,
                  boolean(win),item4,boolean(win),item5,
                  boolean(win),item6)as (win,item)''')

win_df = items_df.filter(items_df.win== 'true')
win_df= win_df.filter(win_df.item != 0)

win_df= win_df.groupBy('item').count()



all_item_df= items_df.filter(items_df.item != 0)
all_item_df = all_item_df.groupBy('item').count().selectExpr('item as item2','count as all_matches_count')

items_winrate_df= all_item_df.join(win_df, all_item_df.item2 == win_df.item, 'outer' )

items_winrate_df = items_winrate_df.withColumn('winrate', (col('count') / col('all_matches_count')) * 100 )\
    .selectExpr('item2','winrate').dropna()

    
items_winrate_df=items_winrate_df.orderBy(items_winrate_df['winrate'].desc())


Item pick Rate

In [407]:
items_pick_rate_df = all_item_df.withColumn('pick_Rate', (all_item_df.all_matches_count / match_count) * 100 )
items_pick_rate_df=items_pick_rate_df.orderBy(items_pick_rate_df['pick_Rate'].desc())



Item Synergies

In [408]:
Champion_item_Synergies= raw_df.selectExpr('''inline(arrays_zip(info.participants.item0,
                        info.participants.item1,info.participants.item2, info.participants.item3,
                        info.participants.item4, info.participants.item5,
                        info.participants.item6,info.participants.win,info.participants.championName))''' , 'metadata.matchId')


Champion_item_Synergies = Champion_item_Synergies.selectExpr('''stack(7,boolean(win),item0,boolean(win),item1,
                  boolean(win),item2,boolean(win),item3,
                  boolean(win),item4,boolean(win),item5,
                  boolean(win),item6)as (win,item)''', 'championName','matchId' )

Champion_item_Synergies= Champion_item_Synergies.filter(Champion_item_Synergies.item != 0)


Champion_item_Synergies_all_df= Champion_item_Synergies.groupBy('championName', 'item')\
    .count().selectExpr('championName as championName1', 'item as item1','count as all_matches_count')


Champion_item_Synergies_win_df = Champion_item_Synergies.filter(Champion_item_Synergies.win=='true')



Champion_item_Synergies_win_df= Champion_item_Synergies_win_df.groupBy('championName', 'item')\
    .count().selectExpr('championName', 'item','count as win_count')



Champion_item_Synergies_win_rate_df = Champion_item_Synergies_win_df.join(Champion_item_Synergies_all_df\
    ,(Champion_item_Synergies_win_df.item==Champion_item_Synergies_all_df.item1)\
        & (Champion_item_Synergies_win_df.championName==Champion_item_Synergies_all_df.championName1)\
            ,'outer')

Champion_item_Synergies_win_rate_df=Champion_item_Synergies_win_rate_df.filter(Champion_item_Synergies_win_rate_df.win_count!=0)   

Champion_item_Synergies_win_rate_df=Champion_item_Synergies_win_rate_df.withColumn('win_Rate', \
    (Champion_item_Synergies_win_rate_df.win_count /Champion_item_Synergies_win_rate_df.all_matches_count) * 100\
         ).selectExpr('championName','item','win_Rate')

Champion_item_Synergies_win_rate_df=Champion_item_Synergies_win_rate_df.orderBy(Champion_item_Synergies_win_rate_df['win_Rate'].desc())


champions with highst win rate per country

In [409]:
country_df= raw_df.selectExpr('inline(arrays_zip(info.participants.championName,info.participants.win))', 'metadata.matchId')

country_df = country_df.withColumn('region', split(country_df.matchId,'_')[0])

country_win_df= country_df.filter(country_df.win=='true').groupBy('region','championName').count()

country_all_df= country_df.groupBy('region','championName').count()\
    .selectExpr('region as region1','championName as championName1', 'count as count1' )


country_most_win_chaps_df= country_win_df.join(country_all_df\
    ,(country_win_df.region==country_all_df.region1)&\
        (country_win_df.championName==country_all_df.championName1)\
            ,'outer')

country_most_win_chaps_df=country_most_win_chaps_df.fillna(0)


country_most_win_chaps_df= country_most_win_chaps_df.withColumn('win_Rate', (col('count') / col('count1')*100)) \
    .selectExpr('championName','region','win_Rate')

country_most_win_chaps_df= country_most_win_chaps_df.filter(country_most_win_chaps_df.win_Rate !=0)
country_most_win_chaps_df=country_most_win_chaps_df.orderBy(country_most_win_chaps_df['win_Rate'].desc())


Writin the analysis results

In [414]:
country_most_win_chaps_df.write.options(header=True).csv('champions_with_highst_win_rate_per_country.csv')

Champion_item_Synergies_win_rate_df.write.options(header=True).csv('Item_Synergies.csv')

items_pick_rate_df.write.options(header=True).csv('Item_pick_Rate.csv')

items_winrate_df.write.options(header=True).csv('Item_win_Rate.csv')

best_Champion_Synergies_df.write.options(header=True).csv('Champion_Synergies.csv')

pick_rate_df.write.options(header='True').csv('Champion_Pick_Rate.csv')

ban_df.write.options(header='True').csv('Champion_Ban_Rate.csv')

win_rate_df.write.options(header='True').csv('Champion_win_Rate.csv')



In [ ]:
import json
with open('items.json') as f:
    items_file = json.load(f)
items= {}
for i in items_file[0:10]:
    items[i['id']]= i['name']